# BDA Assignment #2: MapReduce
## Calculating the Term Frequency (TF), and Inverse Document Frequency (IDF).


#### Group members: 
Aaqib Ahmed Nazir (i22-1920),  
Arhum Khan (i22-1967), 
Ammar Khasif (i22-1968)

##### Section: DS-D   

#### Libraries Useds:

In [1]:
import pandas as pd 
import numpy as np
import os

### Function To Genrate Vocabulary:

In [2]:
# def make_vocabulary(file_path, chunk_size):
#     # divide the file into chunks
#     chunks = pd.read_csv(file_path, chunksize=chunk_size)
#     vocabulary = set()
    
#     for chunk in chunks:
#         for index, row in chunk.iterrows():
#             words = row['SECTION_TEXT'].split()
#             # adding the words to the vocabulary
#             vocabulary.update(words) 
#     return vocabulary


### Function To Save Vocabulary:

In [3]:
# def save_vocabulary(vocabulary, file_path):
#     # save the vocabulary to a file
#     with open(file_path, 'w', encoding='utf-8') as f:
#         for word in vocabulary:
#             f.write(word + '\n')

### Creating a vocabulary from the given dataset and saving it to a file.

In [4]:
# file_path = "/home/arhum/Downloads/preprocessed_dataset.csv"
# chunk_size = 10000
# vocabulary = make_vocabulary(file_path, chunk_size)
# save_vocabulary(vocabulary, "vocabulary.txt")

### For testing purposes, we will use the first 1000 lines of the dataset.

In [5]:
# Taking a chunk of 10000 rows from the dataset
chunk_size = 10000
file_path = "C:\\Users\\Arhum Khan\\Desktop\\preprocessed_dataset.csv"
chunks = pd.read_csv(file_path, chunksize=chunk_size)
chunks = next(chunks)

display(chunks.head())

,ARTICLE_ID,TITLE,SECTION_TITLE,SECTION_TEXT
0,0,Anarchism,Introduction,anarchism political philosophy advocates selfg...
1,0,Anarchism,Etymology and terminology,term anarchism compound word composed word ana...
2,0,Anarchism,History,zzorigins woodcut diggers document william eve...
3,0,Anarchism,Anarchist schools of thought,portrait philosopher pierrejoseph proudhon 180...
4,0,Anarchism,Internal issues and debates,consistent anarchist values controversial subj...


### Function make a vocabulary from the given dataset and save it to a file:

In [6]:
# creating a vocabulary of the chunk
vocabulary = set()
for index, row in chunks.iterrows():
    words = row['SECTION_TEXT'].split()
    for word in words:
        if word not in vocabulary:
            vocabulary.add(word)
            
            
# saving the vocabulary to a file
with open("vocabulary.txt", 'w', encoding='utf-8') as f:
    for word in vocabulary:
        f.write(word + '\n')
        
len(vocabulary)

177860

### Assigning all the words in the vocabulary a unique index:

In [7]:
# creating a dictionary of word to id and id to word
word_to_id = {word: idx for idx, word in enumerate(vocabulary)}
id_to_word = {idx: word for word, idx in word_to_id.items()}

# saving to a csv file
word_to_id_df = pd.DataFrame(list(word_to_id.items()), columns=['word', 'id'])
word_to_id_df.to_csv('Word_IDs.csv', index=False)

### Term Frequency:

Term Frequency (TF) is a measure of how often a word appears in a document. It is calculated as the number of times a word appears in a document divided by the total number of words in the document.

In [21]:
article_word_counts = {}

for index, row in chunks.iterrows():
    # get the word counts for each article
    article_id = row['ARTICLE_ID']
    words = row['SECTION_TEXT'].split()
    word_count = {}
    
    for word in words:
        word_id = word_to_id[word]
        if word_id in word_count:
            word_count[word_id] += 1
        else:
            word_count[word_id] = 1
    # save the word counts for each article
    article_word_counts[article_id] = [(word_id, count) for word_id, count in word_count.items()]
    
# saving to a csv file
article_word_counts_df = pd.DataFrame([(article_id, word_id, count) for article_id, word_counts in article_word_counts.items() for word_id, count in word_counts], columns=['article_id', 'word_id', 'frequency'])
article_word_counts_df.to_csv('TF.csv', index=False)

### IDF:

For a given word, the Inverse Document Frequency (IDF) is the logarithm of the total number of documents divided by the number of documents containing the word.

In [9]:
# Count the number of articles in which each word appears
word_frequency = {}
for article_id, word_counts in article_word_counts.items():
    for word_id, count in word_counts:
        if word_id in word_frequency:
            word_frequency[word_id] += 1
        else:
            word_frequency[word_id] = 1
            
# save the word_frequency dictionary to a csv file with word_id and frequency as columns
word_frequency_df = pd.DataFrame(list(word_frequency.items()), columns=['word_id', 'frequency'])
word_frequency_df.to_csv('IDF.csv', index=False)

#### Calculating the TF-IDF Weights:

In [22]:
# Load the files
tf_df = pd.read_csv("TF.csv")
idf_df = pd.read_csv("IDF.csv")

# Calculate the TF-IDF values
# the TF-IDF score for each word in each article is the term frequency of the word in the whole dataset divided by the IDF of the word
# we need to add the term frequency of the word if it is appearing in 2 different articles then divide by the IDF of the word

tf_idf = []
for index, row in tf_df.iterrows():
    article_id = row['article_id']
    word_id = row['word_id']
    frequency = row['frequency']
    tf = frequency
    idf = idf_df[idf_df['word_id'] == word_id]['frequency'].values[0]
    tf_idf.append((article_id, word_id, tf/idf))
    
# save the tf_idf values to a csv file
tf_idf_df = pd.DataFrame(tf_idf, columns=['article_id', 'word_id', 'tf_idf'])
tf_idf_df.to_csv('TF_IDF.csv', index=False)    